<div style="text-align: right">
<pre>
Author  : Lucas Nelson 
netID   : lln2         
Section : Online       
</pre>
</div>

# STAT 430: Final Project Report

### Table of Contents
1. [Data Section](#Data-Section)
    * [Importing Libraries & Data](#Importing-Libraries-&-Data)
    * [Exploratory Data Analysis](#Exploratory-Data-Analysis)
    * [Building DataFrame](#Building-DataFrame)
    * [Writing to `statcast_pitcher.csv`](#Writing-to-`statcast_pitcher.csv`)
    * [Testing Plotting Functions](#Testing-Plotting-Functions)
2. [Project Report](#Project-Report)
    * [Dataset Description](#Dataset-Description)
    * [Question of Interest](#Question-of-Interest)
    * [Web App Testing](#Web-App-Testing)
    * [Conclusion](#Conclusion)

## Data Section

#### Importing Libraries & Data

In [ ]:
import pandas as pd
import numpy as np
import plotly
from plotly.figure_factory import create_distplot
from pybaseball import statcast, statcast_pitcher, playerid_lookup, playerid_reverse_lookup

In [ ]:
fish_net = statcast(start_dt='2016-01-01', end_dt='2020-12-31')

#### Exploratory Data Analysis

Takeaways from brief EDA:
* `fish_net.shape` returns shape (3.212.937, 92)
* `fish_net.columns` displays variables describing game/season data, player identification data, pitch data, and much more
* `fish_net.dtypes` was unsurprising - thankfully no columns will need to change datatype
* `fish_net.isnull().sum()` showed that some variables contain a couple thousand missing entry points; will keep in mind for data that is selected for final dataframe

In [ ]:
fish_net.shape

In [ ]:
fish_net.columns

In [ ]:
fish_net.dtypes

In [ ]:
fish_net.isnull().sum()

#### Building DataFrame

Selecting covariates of interest before appending data together. Data includes all data for unique pitchers who played at any point from the start of 2016 to the end of 2020.

Covariates of interest:
* `game_year` : season the pitch was recorded; will be used for data selection for the `plotly` plots
* `pitcher_name` : name of pitcher; will be used for data selection for the `plotly` plots
* `pitch_type` : abbreviated name for recorded pitch; will be used for labelling the figures in the `plotly` plots
* `pitch_name` : name for recorded pitch; more detail if necessary
* `release_speed` : speed (mph) recorded pitch is thrown; will be used for graphing data points for specific season-pitcher pair
* `p_throws` : throwing hand of pitcher; will be used for data selection for the `plotly` plots
* `release_spin_rate` : spin rate (rpm) recorded pitch is thrown; will be used for graph data points for specific season-pitcher pair

In [ ]:
keeps = ['game_year', 'player_name', 'pitch_type', 'pitch_name', 'release_speed', 'p_throws', 'release_spin_rate']
appended_data = []

for id in fish_net['pitcher'].unique():
    temp_df = statcast_pitcher(start_dt='2016-01-01',
                               end_dt='2020-12-31',
                               player_id = id)[keeps]
    appended_data.append(temp_df)

In [ ]:
appended_data = pd.concat(appended_data)
appended_data.reset_index(inplace=True, drop=True)
for col in appended_data.columns:
    appended_data = appended_data[appended_data[str(col)].notna()]

#### Writing to `statcast_pitcher.csv`

In [ ]:
appended_data.to_csv('statcast_pitcher.csv')

#### Testing Plotting Functions

Preview of the `pitcher-release-speed` function that I use in the `Dash` application (although `pitcher-release-spin` is an almost identical function). Essentially, it is gathering the `(pitcher, year)` specific data from the defined DataFrame and gathering the `release_speed` data (and `release_spin_rate` data) before graphing the data as a kernel densite estimate plot.

**Important Note**
I have since updated this function within the script that contains my `Dash` application. It no longer contains slicing the `pitcher_name` and does not create a new DataFrame inside of the function (realized how terrible an idea that was just before running it).

In [ ]:
def retrieve_statcast_pitcher(pitcher_name, year):
    pitcher_last = pitcher_name[:pitcher_name.find(',')]
    pitcher_first = pitcher_name[pitcher_name.rfind(', ') + 2:]
    df = statcast_pitcher(start_dt=str(year)+'-01-01',
                        end_dt=str(year)+'-12-31',
                        player_id = playerid_lookup(pitcher_last, pitcher_first)['key_mlbam'][0])
    df = df[['game_year', 'pitch_type', 'release_speed', 'events', 'p_throws']]
    df = df[df['game_year'].notna()]
    df = df[df['pitch_type'].notna()]
    df = df[df['release_speed'].notna()]
    df = df[df['p_throws'].notna()]
    
    kde_data = []
    pitch_labels = []
    for pitch_type in df['pitch_type'].unique():
        if (pitch_type != 'PO') & (pitch_type != 'IN'):
            kde_data.append(df[df['pitch_type'] == pitch_type]['release_speed'])
            pitch_labels.append(pitch_type)
        else:
            continue

    fig = create_distplot(kde_data, pitch_labels, show_hist=False, show_rug=False)
    fig.show()
    
retrieve_statcast_pitcher('Verlander, Justin', 2016)

---

## Project Report

#### Dataset Description

The dataset used for this project originates from Major League Baseball's (MLB) tracking technology known as Statcast. Introduced in 2015, Statcast collects and analyzes immense amounts of baseball data ranging from the spin rate of a Clayton Kershaw curveball to the launch angle of a Giancarlos Stanton homerun. Although Statcast is not MLB's first iteration of an in-game tracking system, it is certainly the most exciting iteration for data-savvy fans and profit-driven front office executives.

To gather the data necessary for this project, I utilized `pybaseball`, a Python package for baseball data analysis. Using package-specific functionality like `statcast()`, `statcast_pitcher()`, and `playerid_lookup()`, I was able to obtain a dataset containing ~3,200,000 observations across 92 variables by gathering all Statcast data generated in the 2016, 2017, 2018, 2019, and 2020 seasons. Since I only worked with pitching data, I gathered the unique instances of pitcher player IDs and, using `statcast_pitcher()`, retrieved a dataset of every pitch thrown by the 1,512 unique pitchers in the aforementioned seasons. After selecting columns specific to the question of interest (see next section), I ended up with an impressive amount of data ready for analysis.

#### Question of Interest

It's not every day I gather data for every unique pitch for five consecutive seasons of MLB gameplay. The data I gathered could explain an array of questions related to a pitcher's change in performance over time or sudden changes in injury trajectory, questions that make or break a pitcher's career in the MLB. For my project, however, I am not responsible for making decisions that cost an organization millions of dollars to make. Rather, I was interested in plotting the distribution of pitch velocity for a specific player given a specific season (similar to Baseball Savant, shown below).

![title](screenshot2.png)

#### Web App Testing

Using `Dash`, I was able to design an application that allows a user to not only observe the pitch velocity distribution of a specific player but also the chance to interact with the application itself. This level of customization grants the user the freedom to select parameters that interest them and the ability to obtain said data in a matter of seconds.

When developing the app, testing was a bit difficult at first since there are many observations within the dataset. After successfully filtering the dataset given the user's provided parameters, there were many challenges that arose. The most obvious challenge was working with missing data. Over the course of an entire season, some pitchers threw upwards of 4000 pitches, yet one missing data point could null that entire season. This challenge was solved for by removing missing data from the filtered data set: `pitcher_data.dropna(inplace=True)`

Another challenge I faced was accounting for all possible parameters the user could provide. Not every pitcher in this dataframe pitched every season the data spans. Therefore, I had to make sure that each season-hand pairing provided a dropdown list that was consistent with the data gathered. Additionally, even though the user had no discretion for this parameter, each pitcher throws a unique rotation of pitch types, and I had to ensure that my code could plot each of these pitch types accurately for all 1500+ possible season-pitcher combinations.

One final challenge I encountered, and arguably the most important one, was presenting the data to the user in a comprehensible manner. Although the app is simply black text over a white background, there were many avenues to explore for customization. Using script-wide properties of `HTML` or arguments unique to `plotly` figures, I utilized bolded and italcized text to attract the user's attention and provided axis labels and legends to describe the data more clearly for the user.

![title](screenshot3.png)

#### Conclusion

As a lifelong fan of baseball and student of statistics, I was ecstatic to analyze and visualize data that you don't necessarily see while spending time at a ballpark (at least, as of right now). Sabermetrics is an exciting field and I'm happy that I was able to contribute to it as a result of this project.

However, there are aspects of this project I could have improved on. Foremost, although I answered the question I asked earlier in my report, I didn't necessarily answer the question asked of me for this project. Rather than adjusting the `year-slider` based on the player selection, I instead required the user to provide a year before selecting a player. Additionally, although to a less severe degree, if time/patience permitted, I would have preferred to provide a slightly darker background to the whole app and provided more customization to the graphs themselves.

Regardless of the successes and challenges I encountered, I thoroughly enjoyed taking on this project. Although I got to work with data I am deeply invested in, I was able to extend my Python capabilities and become more aware of how someone other than myself interacts with my code.